In [2]:
import pandas as pd
from scipy.spatial.distance import cosine
import numpy as np
from scipy.sparse import csr_matrix
from __future__ import division
import matplotlib.pyplot as plt
from scipy import fft, arange, signal
import tensorflow as tf
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
data = pd.read_csv("train_data_400.csv")
test = pd.read_csv("test_data_400.csv")

In [5]:
action_list = ['dws', 'jog', 'ups', 'wlk','sit','std']


#user_id=1
#t_data = data.loc[(data['action_id'].str.match(action_id+"$") ) & (data['user_id'] == user_id)]

total_list = []
curve_acc = 0.5

for action_id in action_list:
  print("-"*50,"\naction_id:",action_id)
  t_data = data.loc[(data['action_id'].str.match(action_id) )]
  t_data = t_data.drop(['action_id','user_id', 'time_50hz'], axis=1)
  t_data.pop(t_data.columns[0]) #remove index


  e = t_data.T
  #e = t_data
  c = np.corrcoef(e)

  e = np.abs(c)
  print(e.shape)
  #print(b)
  #print(*e, sep="\n")
  #print(e.shape)

  curve = curve_acc
  cnt_t = 0

  target = [1]*len(e)

  print(np.around(e, decimals=3))

  for i in range(len(e)):
      cnt = 0
      for j in range(i+1, len(e)):
          if e[i][j] > 1-curve and  target[j] == 1:
              cnt+=1
              target[j] = 0
              cnt_t += cnt
  #    print(cnt, end=", ")


  print("\n", cnt_t)
  print(np.sum(target))
  for i in range(len(target)):
    if target[i]:
      print(t_data.columns[i], end=', ')
  total_list.append(target)

  print("\n","-"*50)
  
print(*total_list, sep="\n")

-------------------------------------------------- 
action_id: dws
(11, 11)
[[1.    0.079 0.115 0.977 0.015 0.009 0.063 0.016 0.034 0.023 0.017]
 [0.079 1.    0.135 0.085 0.005 0.005 0.048 0.019 0.002 0.017 0.019]
 [0.115 0.135 1.    0.193 0.132 0.098 0.04  0.004 0.094 0.034 0.036]
 [0.977 0.085 0.193 1.    0.045 0.008 0.077 0.002 0.03  0.017 0.005]
 [0.015 0.005 0.132 0.045 1.    0.006 0.024 0.207 0.084 0.034 0.284]
 [0.009 0.005 0.098 0.008 0.006 1.    0.079 0.324 0.046 0.08  0.02 ]
 [0.063 0.048 0.04  0.077 0.024 0.079 1.    0.065 0.019 0.019 0.064]
 [0.016 0.019 0.004 0.002 0.207 0.324 0.065 1.    0.07  0.033 0.133]
 [0.034 0.002 0.094 0.03  0.084 0.046 0.019 0.07  1.    0.073 0.124]
 [0.023 0.017 0.034 0.017 0.034 0.08  0.019 0.033 0.073 1.    0.078]
 [0.017 0.019 0.036 0.005 0.284 0.02  0.064 0.133 0.124 0.078 1.   ]]

 1
10
attitude.pitch, attitude.yaw, gravity.x, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.

In [6]:
t_data

,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z
84000,-1.051363,1.611548,0.496386,0.868100,0.001555,-0.027765,-0.014517,0.096305,0.019346,0.030886,0.008607
84001,-1.051097,1.611145,0.496618,0.867968,0.001137,-0.031915,-0.006982,0.079159,-0.069464,0.063764,-0.021080
84002,-1.050255,1.612825,0.497346,0.867550,0.001825,-0.041191,0.001830,0.014990,-0.073061,0.046253,-0.026514
84003,-1.053220,1.614236,0.494771,0.869021,0.001949,-0.073207,-0.057491,-0.041800,-0.037023,-0.020236,0.048161
84004,-1.054274,1.618206,0.493848,0.869542,0.003291,-0.103940,-0.024308,-0.054876,-0.022626,-0.037312,0.048344
84005,-1.054344,1.624517,0.493762,0.869576,0.006022,-0.146610,-0.008187,-0.047686,0.023450,-0.003472,0.015142
84006,-1.055112,1.631102,0.493054,0.869955,0.008673,-0.164824,-0.017740,-0.040323,0.044559,0.063455,-0.041068
84007,-1.055771,1.637890,0.492417,0.870280,0.011753,-0.175301,0.035518,-0.029782,0.048583,0.065068,-0.057133
84008,-1.056432,1.651168,0.491626,0.870606,0.018687,-0.206295,0.065316,0.007345,-0.012607,0.011581,0.062459
84009,-1.055952,1.657187,0.491911,0.870369,0.021918,-0.111402,0.067055,0.022960,-0.036482,0.023597,0.013650
